In [1]:
import tensorflow as tf
import numpy as np
from utils import *
from VDSH_S import *
from VDSH_SP import *
from __future__ import print_function

filename = 'dataset/ng20.tfidf.mat'
data = Load_Dataset(filename)

latent_dim = 32
sess = get_session("1", 0.10)
model = VDSH_S(sess, latent_dim, data.n_feas, data.n_tags, use_cross_entropy=True)

In [2]:
# create an optimizer
learning_rate=0.001
decay_rate = 0.96
#decay_step = 10000
step = tf.Variable(0, trainable=False)  
lr = tf.train.exponential_decay(learning_rate, 
                                step, 
                                10000, 
                                decay_rate, 
                                staircase=True, name="lr")

my_optimizer = tf.train.AdamOptimizer(learning_rate=lr) \
                     .minimize(model.cost, global_step=step)
    
init = tf.global_variables_initializer()
model.sess.run(init)

In [3]:
total_epoch = 15
kl_weight = 0.
kl_inc = 1 / 5000. # set the annealing rate for KL loss

pred_weight = 0.
pred_inc = 0.1
max_pred_weight = 150.

for epoch in xrange(total_epoch):
    epoch_loss = []
    for i in range(len(data.train)):
        # get doc
        doc = data.train[i]
        labels = data.gnd_train[i]
        word_indice = np.where(doc > 0)[0]
        
        # indices
        opt, loss = model.sess.run((my_optimizer, model.cost), 
                                    feed_dict={model.input_bow: doc.reshape((-1, data.n_feas)),
                                               model.input_bow_idx: word_indice,
                                               model.labels: labels.reshape((-1, data.n_tags)),
                                               model.kl_weight: kl_weight,
                                               model.tag_weight: pred_weight,
                                               model.keep_prob: 0.9})
        
        kl_weight = min(kl_weight + kl_inc, 1.0)
        pred_weight = min(pred_weight + pred_inc, max_pred_weight)

        epoch_loss.append(loss[0])
        
        if i % 50 == 0:
            print("\rEpoch:{}/{} {}/{}: Loss:{:.3f} AvgLoss:{:.3f}".format(epoch+1, total_epoch, i, 
                                                                           data.n_trains, loss[0], np.mean(epoch_loss)), end='')
            
    # run experiment on CV dataset
    zTrain = model.transform(data.train)
    zCV = model.transform(data.cv)
    zTrain = np.array(zTrain)
    zCV = np.array(zCV)
    medHash = MedianHashing()
    cbTrain = medHash.fit_transform(zTrain)
    cbCV = medHash.transform(zCV)

    TopK=100
    print('Retrieve Top{} candidates using hamming distance'.format(TopK))
    results = run_topK_retrieval_experiment(cbTrain, cbCV, data.gnd_train, data.gnd_cv, TopK)
    avg_prec_at_k, avg_recall_at_k, avg_ndcg

Epoch:1/15 10950/11016: Loss:7103.933 AvgLoss:1506.846

  1%|          | 108/11016 [00:00<00:10, 1074.66it/s]

Epoch:1/15 11000/11016: Loss:9595.529 AvgLoss:1538.058

100%|██████████| 3667/3667 [00:02<00:00, 1437.68it/s]


Retrieve Top100 candidates using hamming distance


...

Epoch:15/15 11000/11016: Loss:6161.544 AvgLoss:819.146

100%|██████████| 3667/3667 [00:03<00:00, 1063.31it/s]


Retrieve Top100 candidates using hamming distance


100%|██████████| 3667/3667 [01:22<00:00, 44.48it/s]


Prec@K = 0.7545, Recall@K = 0.1351, NDCG@K = 0.8020


In [4]:
# run experiment here
zTrain = model.transform(data.train)
zTest = model.transform(data.test)
zTrain = np.array(zTrain)
zTest = np.array(zTest)
medHash = MedianHashing()
cbTrain = medHash.fit_transform(zTrain)
cbTest = medHash.transform(zTest)

TopK=100
print('Retrieve Top{} candidates using hamming distance'.format(TopK))
results = run_topK_retrieval_experiment(cbTrain, cbTest, data.gnd_train, data.gnd_test, TopK)

100%|██████████| 3668/3668 [00:03<00:00, 1003.25it/s]


Retrieve Top100 candidates using hamming distance


100%|██████████| 3668/3668 [01:20<00:00, 45.41it/s]


Prec@K = 0.7475, Recall@K = 0.1339, NDCG@K = 0.7973
